<a href="https://colab.research.google.com/github/talhaanwarch/mia-covid19/blob/main/ag2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-32e4e532-0d2d-6b35-0b72-5e695e87edf6)


In [2]:
%%capture 
!pip install -U  pip
!pip install -U setuptools wheel
!pip install -U "mxnet_cu110<2.0.0"
!pip install  autogluon  # autogluon==0.2.0

# Get data from drive

In [3]:
#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/covid/train.zip /content/
!cp /content/drive/MyDrive/covid/val.zip /content/

In [5]:
%%capture 
!unzip /content/train.zip
!rm /content/train.zip
!mv /content/resized /content/train


In [6]:
%%capture 
!unzip /content/val.zip -d val
!rm /content/val.zip


# import packages

In [7]:
import pandas as pd
import numpy as np
from glob import glob
import cv2
import pandas as pd
import autogluon.core as ag
from autogluon.vision import ImagePredictor
from sklearn.metrics import classification_report


# prepare data

In [8]:
# #load data, data loaded is in format of list of list containing folder and image paths
# train_pos_path=[glob(folder+'*.jpg') for folder in glob('train/covid/*/')]
# train_neg_path=[glob(folder+'*.jpg') for folder in glob('train/non-covid/*/')]
# len(train_pos_path),len(train_neg_path)

In [9]:
# #create labels for list of list images
# train_pos_label=[[0]*len(i)  for i in train_pos_path]
# train_neg_label=[[1]*len(i)  for i in train_neg_path]
# len(train_pos_label),len(train_neg_label)

In [10]:
# #combine data
# train_path=train_pos_path+train_neg_path
# train_label=train_pos_label+train_neg_label
# len(train_path),len(train_label)

In [11]:
# #plit data 
# from sklearn.model_selection import train_test_split
# train_path,val_path,train_label,val_label=train_test_split(train_path,train_label,test_size=0.2)

In [12]:
# #flatten list
# train_path = [item for sublist in train_path for item in sublist]
# val_path = [item for sublist in val_path for item in sublist]
# train_label = [item for sublist in train_label for item in sublist]
# val_label = [item for sublist in val_label for item in sublist]
# len(train_path),len(train_label),len(val_path),len(val_label)

In [13]:
# train_df=pd.DataFrame(zip(train_path,train_label),columns=['img','label'])
# train_df = train_df.sample(frac=1).reset_index(drop=True)
# train_df.to_csv('train_df.csv',index=False)
# train_df.head()

In [14]:
# val_df=pd.DataFrame(zip(val_path,val_label),columns=['img','label'])
# val_df = val_df.sample(frac=1).reset_index(drop=True)
# val_df.to_csv('val_df.csv',index=False)
# val_df.head()

In [ ]:
#create test data
test_pos_path=[glob(folder+'*.jpg') for folder in glob('val/covid/*/')]
test_neg_path=[glob(folder+'*.jpg') for folder in glob('val/non-covid/*/')]
test_pos_label=[[0]*len(i)  for i in test_pos_path]
test_neg_label=[[1]*len(i)  for i in test_neg_path]
print('folder images', len(test_pos_path),len(test_neg_path))
print('folder label', len(test_pos_label),len(test_neg_label))
#combine data
test_path=test_pos_path+test_neg_path
test_label=test_pos_label+test_neg_label
len(test_path),len(test_label)
#flatten list
test_path = [item for sublist in test_path for item in sublist]
test_label = [item for sublist in test_label for item in sublist]
len(test_path),len(test_label)
#convert to dataframe
test_df=pd.DataFrame(zip(test_path,test_label),columns=['image','label'])
test_df = test_df.sample(frac=1).reset_index(drop=True)
test_df.to_csv('test_df.csv',index=False)
test_df.head()

In [15]:
train_df=pd.read_csv('/content/drive/MyDrive/covid/train_df.csv')
train_df.columns=['image','label']
val_df=pd.read_csv('/content/drive/MyDrive/covid/val_df.csv')
val_df.columns=['image','label']
val_df.to_csv('val_df.csv',index=False)
print(len(train_df),len(val_df))
train_df.head()

266607 69065


,image,label
0,train/non-covid/ct_scan_790/80.jpg,1
1,train/non-covid/ct_scan_580/36.jpg,1
2,train/covid/ct_scan_11/122.jpg,0
3,train/non-covid/ct_scan_199/280.jpg,1
4,train/non-covid/ct_scan_530/234.jpg,1


# train model

In [ ]:
# %%time
# predictor = ImagePredictor(verbosity=1)
# predictor.fit(train_df,tuning_data=val_df, hyperparameters={'epochs': 5}) 

In [ ]:
# fit_result = predictor.fit_summary()
#(fit_result)
# print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

In [18]:
#test here i am using validation data provided as test
#1:pos,covid, 0:neg,non-covid
result = predictor.predict('train/non-covid/ct_scan_790/80.jpg')
result

0    1
Name: label, dtype: int64

In [ ]:
val_df=ImagePredictor.Dataset.from_csv('/content/val_df.csv')
res = predictor.evaluate(val_df)
print('Top-1 test acc: %.3f' % res[0])

In [ ]:
test_df=ImagePredictor.Dataset.from_csv('/content/test_df.csv')
res = predictor.evaluate(test_df)
print('Top-1 test acc: %.3f' % res[0])

In [ ]:
test_pred=predictor.predict(test_df)
print(classification_report(test_df.label,test_pred))

In [16]:
predictor = ImagePredictor.save('/content/drive/MyDrive/covid/predictor.ag')


INFO:root:Model file not found. Downloading.


100%|██████████| 55344/55344 [00:02<00:00, 24935.52KB/s]


# final evaluation on validation data

In [33]:
#now loop each folder of each directoy and create a dataframe for each folder
def evaluate(path,label):
  pos_test_pred=[]
  for i, j in zip(path,label):
    test_df=pd.DataFrame(zip(i,j),columns=['image','label'])
    test_df.to_csv('test_df.csv',index=False)
    test_df=ImagePredictor.Dataset.from_csv('/content/test_df.csv')
    pred=list(predictor.predict(test_df))
    #find maximum occurent of element
    pos_test_pred.append(max(pred,key=pred.count))
    return pos_test_pred



165 165


In [34]:
neg_test_pred=[]
for i, j in zip(test_neg_path,test_neg_label):
  test_df=pd.DataFrame(zip(i,j),columns=['image','label'])
  test_df.to_csv('test_df.csv',index=False)
  test_df=ImagePredictor.Dataset.from_csv('/content/test_df.csv')
  pred=list(predictor.predict(test_df))
  #find maximum occurent of element
  neg_test_pred.append(max(pred,key=pred.count))
print(len(test_neg_path),len(neg_test_pred))  

209 209


In [35]:
test_true=test_pos_label+test_neg_label
test_true=[int(np.mean(i)) for i in test_true]
test_pred=pos_test_pred+neg_test_pred
print(len(test_true),len(test_pred))  

374 374


In [36]:
from sklearn.metrics import classification_report
print(classification_report(test_true,test_pred))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85       165
           1       0.87      0.91      0.89       209

    accuracy                           0.87       374
   macro avg       0.87      0.87      0.87       374
weighted avg       0.87      0.87      0.87       374

